In [16]:
import sys
import pandas as pd
import numpy as np
import requests
import string
import json
import datetime
import sqlite3
from pandas.core.ops import invalid_comparison

#web
import warnings
warnings.filterwarnings('ignore')

import ipywidgets
from IPython.display import display, clear_output

In [10]:
#!jupyter nbextension enable – py widgetsnbextension – sys-prefix
#!jupyter serverextension enable voila – sys-prefix

In [46]:

tab_contents = ['mkdir', 'ls', 'cat', 'rm', 'put', 'getPartitionLocations', 'readPartition']
children = [widgets.Text(value = '',
                         placeholder ='type here',
                         description=name) for name in tab_contents]
tab = widgets.Tab()
tab.children = children

tab.titles = ['mkdir', 'ls', 'cat', 'rm', 'put', 'getPartitionLocations', 'readPartition']
tab

In [17]:
# SQLite set up
edfs = sqlite3.connect("edfs.db")
cur = edfs.cursor()
cur.execute("CREATE TABLE meta(id INTEGER primary key, name TEXT, type TEXT)") 
cur.execute("INSERT INTO meta VALUES (1,'/','d')") # root dir
# meta table stores the id for each file and dir
# Do we need to store permision, access and modification time?
# assign id in order, use max()+1 to assign the id for the next dir/file
cur.execute("CREATE TABLE dir(parent INTEGER, child INTEGER, primary key (parent, child))") 
# dir table stores the directory structure of the file system
cur.execute("CREATE TABLE par(id INTEGER, location)") 
# par table stores the partition infor about a file

In [37]:
def exist(path): # path: directory or file, eg. "/user/john" or "/user/hello.txt"
    path = path.strip() # remove white space before or after the path
    if path[0] == "/" and path[-1] != "/":# valid path starts with "/" - root, and does not end with "/"
    #id_parent = cur.execute("select id from meta where name = '/'").fetchall()  id of root
        id_parent = 1
        path = path[1:] # get rid of the root
        path = path.split("/") # list of the rest of the path - the name of each directory/file
        n = len(path)
        while n > 0: 
            position = len(path)-n
            id_child = cur.execute("select child from dir where parent = ? and child in (select id FROM meta where name = ?)", (id_parent, path[position])).fetchall()
          # check if the child exist and if the the parent-child relationship exist
            if id_child != []:# if there exists such child, it should be unique
                id_parent = id_child[0][0] # set the child as the new parent, and go back to the while loop to check for the next directory/file in path
                n = n - 1
            else:
                break
        if n > 0:
            return ""
        else: 
            return id_parent
    elif path == "/": # if the input is the root
        return 1
    else: 
        return ""

In [43]:
d = ipywidgets.Text(value = '',
                      placeholder ='type here',
                      description = 'mkdir',
                      disabled = False)
instructions = ipywidgets.widgets.HTML('Create a directory in file system(eg. /user/john)')
display(instructions)
display(d)

def mkdir(d):# d: directory
  # if the directory already exists, notify the user
  # check if the dir exist: get the id for the parent dir, search it in the dir table to get all the children
  # use a for loop to go through every child's name in meta table using the ids
    d = str(d.value)
    if d == "/":
        print("Directory already exist.")
    elif d[0] == "/" and d[-1] != "/":
        d = d.split('/')
        d_new = d # copy the list of directories into d_new
        d_new = d_new[:-1]
        if d_new == ['']: 
            d_new = "/"
        else: 
            d_new = "/".join(d_new)
        id_parent = exist(d_new)
        if id_parent != "": 
            id_child = cur.execute("select child from dir where parent = ? and child in (select id FROM meta where name = ?)", (id_parent, d[-1])).fetchall()
            if id_child == []: # check if the directory already exist in the system
                max_id = cur.execute("SELECT max(id) FROM meta").fetchone()
                max_id = max_id[0]
                id_child = max_id + 1 # the id assigned to the new directory
                cur.execute("INSERT INTO meta VALUES (?,?,'d')",(id_child, d[-1])) ##id needs to be defined
                cur.execute("INSERT INTO dir VALUES (?,?)",(id_parent,id_child))
            else: 
                print("Directory already exist.")
        else: 
            print("Please enter a valid directory.")
    else:
        print("Please enter a valid directory.")

d.on_submit(mkdir)


HTML(value='Create a directory in file system(eg. /user/john)')

Text(value='', description='mkdir', placeholder='type here')

In [44]:
cur.execute("Select * from meta")